In [ ]:
import langchain, langgraph, langchain_openai, langsmith

import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain.callbacks.tracers.langchain import LangChainTracer

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['LANGSMITH_API_KEY'] = LANGSMITH_API_KEY
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_ENDPOINT'] = "https://api.smith.langchain.com"
langsmith_project_name = "db_agent_v1"
os.environ['LANGSMITH_PROJECT'] = langsmith_project_name

# Set up LangSmith tracer manually
tracer = LangChainTracer(project_name=langsmith_project_name)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o',temperature=0)

In [4]:
from db_agent_v1.ipynb import objects_documentation


ModuleNotFoundError: No module named 'db_agent_v1'

In [ ]:
objects_documentation

testing_dataset = 
[
    {'input': {'objects_documentation':objects_documentation,
               'messages_log': [],
               'question':question,
               'sql_query': [],
               'sql_query_result': [],
               'sql_query_explanation': [],
               'llm_answer': ''
               },
}]

In [ ]:
# create testing datasets

from langsmith import Client
client = Client()
dataset = client.create_dataset('QA Example Dataset')